In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import Common_Functions as cmn
import math
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [5]:
def init_df():
    df_MAIN = pd.read_csv(cmn.PATH_df_no_NAN_99p)
    df_MAIN_cleaned = df_MAIN.dropna()

    return df_MAIN_cleaned

In [6]:

def CustomRegression_upper(df, size_list, condition_list):

    # load dataframe with all data
    current_df = df    
    #dictionary of results corresponding to each condition
    perCondition = {}

    #takes each condition and 
    for condition in condition_list:
        #create dataframe to house results
        results = pd.DataFrame({'number': size_list, 'mse': [0]*len(size_list), 'r2': [0]*len(size_list)})
        for idx in range (len(size_list)):
            results['number'][idx] = size_list[idx]
            top_n_df = current_df.loc[current_df.groupby('Region of Headquarters')[condition].nlargest(size_list[idx]).reset_index()['level_1']]
            #top_n_df = current_df.groupby('Region of Headquarters').apply(lambda x: x.nlargest(size_list[idx], condition)).reset_index(drop=True)
            top_n_df_reg = top_n_df[[   'Total Return',
                                        'Revenue - Mean',
                                        'Earnings Per Share - Mean',
                                        'Return On Equity - Mean',
                                        'Volume',
                                        'Company Market Cap',
                                        'EBITDA - Mean',
                                        'ESG Score']]
            X = top_n_df_reg.drop('ESG Score',axis =1)
            y = top_n_df_reg['ESG Score']
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 101)
            regr = RandomForestRegressor(n_estimators = 10, max_depth = 10, random_state = 101)
            regr.fit(X_train, y_train.values.ravel())
            predictions = regr.predict(X_test)
            # Mean squared error (MSE)
            results['mse'][idx] = mean_squared_error(y_test.values.ravel(), predictions)
            # R-squared scores
            results['r2'][idx] = r2_score(y_test.values.ravel(), predictions)

        perCondition[condition] = results
    
    return perCondition
        
    


In [7]:
n = [5, 10, 20, 50, 100, 150, 200, 250]
conditioning_feature = ['Company Market Cap', 'Return On Equity - Mean', 'EBITDA - Mean']
df = init_df()

results_ = CustomRegression_upper(df, n, conditioning_feature)
results_

C:\Users\ianpe\AppData\Local\Temp\ipykernel_14516\1953818744.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['r2'][idx] = r2_score(y_test.values.ravel(), predictions)
C:\Users\ianpe\AppData\Local\Temp\ipykernel_14516\1953818744.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['number'][idx] = size_list[idx]
C:\Users\ianpe\AppData\Local\Temp\ipykernel_14516\1953818744.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

{'Company Market Cap':    number         mse        r2
 0       5  128.315233  0.595515
 1      10   99.655351  0.811140
 2      20  270.746706  0.343665
 3      50  152.065144  0.537682
 4     100  238.698296  0.423042
 5     150  218.153326  0.398164
 6     200  300.603727  0.270050
 7     250  256.749100  0.188513,
 'Return On Equity - Mean':    number         mse        r2
 0       5  161.978018 -0.065468
 1      10   88.187451  0.714667
 2      20  167.027133  0.615874
 3      50  241.793046  0.360024
 4     100  235.257143  0.359118
 5     150  256.975286  0.384359
 6     200  218.958082  0.374747
 7     250  268.149204  0.324127,
 'EBITDA - Mean':    number         mse        r2
 0       5   56.000920  0.808580
 1      10  142.031350  0.523026
 2      20  138.850384  0.372646
 3      50  156.313027  0.631127
 4     100  267.171078  0.363411
 5     150  202.554826  0.407067
 6     200  254.200500  0.339509
 7     250  256.742479  0.326383}

In [25]:
def CustomRegression_all(df, size_list, condition_list):

    # load dataframe with all data
    current_df = df    
    #dictionary of results corresponding to each condition
    perCondition = {}

    #takes each condition and 
    for condition in condition_list:
        #create dataframe to house results
        results = pd.DataFrame({'number': size_list, 'mse': [0]*len(size_list), 'r2': [0]*len(size_list)})
        for idx in range (len(size_list)):
            results['number'][idx] = size_list[idx]
            top_n_df = current_df.nlargest(size_list[idx], condition).reset_index(drop=True)
            top_n_df_reg = top_n_df[[   'Total Return',
                                        'Revenue - Mean',
                                        'Earnings Per Share - Mean',
                                        'Return On Equity - Mean',
                                        'Volume',
                                        'Company Market Cap',
                                        'EBITDA - Mean',
                                        'ESG Score']]
            X = top_n_df_reg.drop('ESG Score',axis =1)
            y = top_n_df_reg['ESG Score']
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 101)
            regr = RandomForestRegressor(n_estimators = 10, max_depth = 10, random_state = 101)
            regr.fit(X_train, y_train.values.ravel())
            predictions = regr.predict(X_test)
            # Mean squared error (MSE)
            results['mse'][idx] = mean_squared_error(y_test.values.ravel(), predictions)
            # R-squared scores
            results['r2'][idx] = r2_score(y_test.values.ravel(), predictions)

        perCondition[condition] = results
    
    return perCondition

In [26]:
n = [50, 100, 200, 500, 1000, 5000]
conditioning_feature = ['Company Market Cap', 'Return On Equity - Mean', 'EBITDA - Mean']
df = init_df()

results_ = CustomRegression_all(df, n, conditioning_feature)


C:\Users\ianpe\AppData\Local\Temp\ipykernel_18816\1501184855.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['r2'][idx] = r2_score(y_test.values.ravel(), predictions)
C:\Users\ianpe\AppData\Local\Temp\ipykernel_18816\1501184855.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['number'][idx] = size_list[idx]
C:\Users\ianpe\AppData\Local\Temp\ipykernel_18816\1501184855.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [27]:
for key, result in results_.items():
    print(key)
    print(result)

Company Market Cap
   number         mse        r2
0      50  342.233098  0.344520
1     100  137.165782  0.663510
2     200  232.578795  0.548681
3     500  305.558921  0.265015
4    1000  351.227480  0.243776
5    5000  298.562052  0.218835
Return On Equity - Mean
   number         mse        r2
0      50  480.670084  0.252598
1     100  490.873545  0.058157
2     200  332.009360  0.282678
3     500  263.522971  0.306744
4    1000  322.068481  0.230741
5    5000  288.682634  0.270941
EBITDA - Mean
   number         mse        r2
0      50  151.443863  0.628971
1     100  202.452337  0.540033
2     200  309.410272  0.262252
3     500  334.312162  0.265556
4    1000  327.625511  0.150863
5    5000  312.381477  0.204043


In [23]:

df_change = pd.read_csv(cmn.PATH_df_no_NAN_99p_YoY).drop(columns="Unnamed: 0")
df_change

,ISIN,Year,Total Return,ESG Score,ESG Combined Score,ESG Controversies Score,Social Pillar Score,Governance Pillar Score,Environmental Pillar Score,Revenue - Mean,...,ESG Combined Score YoY change,ESG Controversies Score YoY change,Social Pillar Score YoY change,Governance Pillar Score YoY change,Environmental Pillar Score YoY change,Revenue - Mean YoY change,Earnings Per Share - Mean YoY change,Volume YoY change,Company Market Cap YoY change,EBITDA - Mean YoY change
0,CNE100002RY5,2020,29.320924,61.088930,61.088930,100.0,64.194818,57.173361,61.605232,9.785651e+09,...,0.397965,0.0,0.313728,0.059345,1.265436,0.829777,0.505680,1.985836,0.278810,0.638345
1,CNE100002RY5,2021,-60.719039,62.422402,62.422402,100.0,66.186509,56.906009,63.845722,1.465000e+10,...,0.021828,0.0,0.031026,-0.004676,0.036369,0.497090,0.376078,1.492954,-0.588241,0.389547
2,KYG2953R1149,2014,13.060541,70.872954,70.872954,100.0,83.020173,87.955713,47.609687,8.921968e+09,...,0.779032,0.0,2.294247,0.173571,0.902267,0.068803,-0.012776,-0.645916,0.103586,0.084438
3,KYG2953R1149,2015,24.254758,73.798418,73.798418,100.0,85.473982,88.044625,53.110161,1.155601e+10,...,0.041278,0.0,0.029557,0.001011,0.115533,0.295231,0.268559,-0.179839,0.216606,0.260367
4,KYG2953R1149,2016,41.855474,74.061944,74.061944,100.0,92.580961,87.511979,48.283248,1.494844e+10,...,0.003571,0.0,0.083148,-0.006050,-0.090885,0.293564,0.258486,-1.000000,0.393670,0.295898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23386,TW0001216000,2016,0.331311,82.281792,82.281792,100.0,76.302437,83.142620,90.891038,4.178999e+11,...,0.138407,0.0,0.263931,0.018486,0.102000,-0.016542,0.031223,-1.000000,-0.027322,-0.010988
23387,FR0013326246,2007,-16.521731,73.652381,73.652381,100.0,70.885382,79.110106,71.201105,7.449527e+08,...,2.011378,0.0,1.247467,0.968439,12.498543,0.422394,0.148572,inf,0.440895,0.595658
23388,FR0013326246,2008,-25.198536,84.186496,84.186496,100.0,83.021801,83.587662,85.637085,1.421309e+09,...,0.143025,0.0,0.171212,0.056599,0.202749,0.907919,0.123100,0.126682,-0.286982,0.606961
23389,FR0013326246,2009,52.077456,84.563936,84.563936,100.0,69.027935,93.087604,89.760716,1.469237e+09,...,0.004483,0.0,-0.168557,0.113652,0.048152,0.033721,0.085109,-0.275298,0.606023,0.064199


In [24]:
def prediction_int(df, feature):
    pred = ((df[feature] > 0).astype(int))
    return pred.astype(int)

In [25]:

feature_cols = df_change.filter(regex='Return|YoY').columns.tolist()
print(feature_cols)
df_change_with_classifier = df_change.copy()
for feature in feature_cols:
    df_change_with_classifier[f"{feature} Classified"] = prediction_int(df_change, feature)
df_change_with_classifier

['Total Return', 'Return On Equity - Mean', 'ESG Score YoY change', 'ESG Combined Score YoY change', 'ESG Controversies Score YoY change', 'Social Pillar Score YoY change', 'Governance Pillar Score YoY change', 'Environmental Pillar Score YoY change', 'Revenue - Mean YoY change', 'Earnings Per Share - Mean YoY change', 'Volume YoY change', 'Company Market Cap YoY change', 'EBITDA - Mean YoY change']


,ISIN,Year,Total Return,ESG Score,ESG Combined Score,ESG Controversies Score,Social Pillar Score,Governance Pillar Score,Environmental Pillar Score,Revenue - Mean,...,ESG Combined Score YoY change Classified,ESG Controversies Score YoY change Classified,Social Pillar Score YoY change Classified,Governance Pillar Score YoY change Classified,Environmental Pillar Score YoY change Classified,Revenue - Mean YoY change Classified,Earnings Per Share - Mean YoY change Classified,Volume YoY change Classified,Company Market Cap YoY change Classified,EBITDA - Mean YoY change Classified
0,CNE100002RY5,2020,29.320924,61.088930,61.088930,100.0,64.194818,57.173361,61.605232,9.785651e+09,...,1,0,1,1,1,1,1,1,1,1
1,CNE100002RY5,2021,-60.719039,62.422402,62.422402,100.0,66.186509,56.906009,63.845722,1.465000e+10,...,1,0,1,0,1,1,1,1,0,1
2,KYG2953R1149,2014,13.060541,70.872954,70.872954,100.0,83.020173,87.955713,47.609687,8.921968e+09,...,1,0,1,1,1,1,0,0,1,1
3,KYG2953R1149,2015,24.254758,73.798418,73.798418,100.0,85.473982,88.044625,53.110161,1.155601e+10,...,1,0,1,1,1,1,1,0,1,1
4,KYG2953R1149,2016,41.855474,74.061944,74.061944,100.0,92.580961,87.511979,48.283248,1.494844e+10,...,1,0,1,0,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23386,TW0001216000,2016,0.331311,82.281792,82.281792,100.0,76.302437,83.142620,90.891038,4.178999e+11,...,1,0,1,1,1,0,1,0,0,0
23387,FR0013326246,2007,-16.521731,73.652381,73.652381,100.0,70.885382,79.110106,71.201105,7.449527e+08,...,1,0,1,1,1,1,1,1,1,1
23388,FR0013326246,2008,-25.198536,84.186496,84.186496,100.0,83.021801,83.587662,85.637085,1.421309e+09,...,1,0,1,1,1,1,1,1,0,1
23389,FR0013326246,2009,52.077456,84.563936,84.563936,100.0,69.027935,93.087604,89.760716,1.469237e+09,...,1,0,0,1,1,1,1,0,1,1
